<a href="https://colab.research.google.com/github/Inishpy/ML-and-DL-Project-works/blob/main/DeepQlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


In [ ]:
env = gym.make("CartPole-v0")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns 

In [ ]:
state_size = env.observation_space.shape[0]
state_size

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


4

In [ ]:
action_size = env.action_space.n
action_size

2

In [ ]:
batch_size = 32

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
n_episodes = 1001

In [ ]:
output_dir = 'model_output/cartpole'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Define Agent

In [ ]:
class DQNAgent:

  def __init__(self, state_size, action_size):
    self.state_size = state_size
    self.action_size = action_size

    self.memory = deque(maxlen=2000)
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_decay = 0.995
    self.epsilon_min = 0.01

    self.learning_rate = 0.001
    self.model = self._build_model()

  def _build_model(self):
    model = Sequential()

    model.add(Dense(24, input_dim = self.state_size, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(self.action_size, activation='linear'))

    model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))

    return model

  def remember(self, state, action, reward, next_state, done):
    self.memory.append((state, action, reward, next_state, done))

  def act(self, state):
    if np.random.rand() <= self.epsilon:
      return random.randrange(self.action_size)

    act_values = self.model.predict(state)
    return np.argmax(act_values[0])

  def replay(self, batch_size):

    minibatch = random.sample(self.memory, batch_size)

    for state, action, reward, next_state, done in minibatch:
      target = reward
      if not done:
        target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))


      target_f = self.model.predict(state)
      target_f[0][action] = target

      self.model.fit(state, target_f, epochs=1, verbose=0)

    if self.epsilon > self.epsilon_min:
      self.epsilon *= self.epsilon_decay

  def load(self, name):
    self.model.load_weights(name)

  def save(self, name):
    self.model.save_weights(name)





In [ ]:
agent = DQNAgent(state_size, action_size)

Interact with Environment

In [ ]:
done = False
for e in range(n_episodes):
  state = env.reset()
  state = np.reshape(state, [1, state_size])

  for time in range(5000):

    action = agent.act(state)
    next_state, reward, done, _ = env.step(action)
    reward = reward if not done else -10
    next_state = np.reshape(next_state, [1, state_size])
    agent.remember(state, action, reward, next_state, done)
    state = next_state

    if done:
      print("episode: {}/{}, scores: {}, e: {:.2}".format(e, n_episodes, time, agent.epsilon))
      break

  if len(agent.memory) > batch_size:
    agent.replay(batch_size)


  if e % 50 == 0:
    agent.save(output_dir + "weights_" + "{:04d}".format(e) + ".hdf5")




Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 28ms/step
episode: 306/1001, scores: 37, e: 0.2
1/1 [==============================] - 0s 25ms/step
episode: 307/1001, scores: 46, e: 0.2
1/1 [==============================] - 0s 29ms/step
episode: 308/1001, scores: 28, e: 0.2
1/1 [==============================] - 0s 23ms/step
episode: 309/1001, scores: 54, e: 0.2
1/1 [==============================] - 0s 27ms/step
episode: 310/1001, scores: 44, e: 0.2
1/1 [==============================] - 0s 35ms/step
episode: 311/1001, scores: 45, e: 0.2
1/1 [==============================] - 0s 23ms/step
episode: 312/1001, scores: 54, e: 0.2
1/1 [==============================] - 0s 26ms/step
episode: 313/1001, scores: 49, e: 0.2
1/1 [==============================] - 0s 24ms/step
episode: 314/1001, scores: 112, e: 0.2
1/1 [==============================] - 0s 30ms/step
episode: 315/1001, scores: 63, e: 0.2
1/1 [==============================] - 0s 24ms/s